# Label classifier (3dshapes): data collection

## Task 1

**Author**: Maleakhi A. Wijaya  
**Description**: This notebook contains code used to collect experimentation data. We compare the performance of methods discussed in Rabanset et al. against our proposed CBSD method. The end-to-end task for task 1 is to predict the scale given the 3dshapes images.

In [1]:
# Load utilities functions
%run ../../scripts/constants.py
%run ../../scripts/3dshapes_utils.py
%run ../../scripts/shift_applicator.py
%run ../../scripts/shift_dimensionality_reductor.py
%run ../../scripts/experiment_utils.py
%run ../../scripts/shift_statistical_test.py

In [2]:
## Random seed
SEED = 20
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Load dataset

In [5]:
files_dir = "../../data/3dshapes.h5"
# index 0 = image category
X_train, X_test, y_train, y_test, c_train, c_test = train_test_split_3dshapes(files_dir, 70000, DatasetTask.Task1, 
                                                                               train_size=0.80, class_index=3)

Training samples: 56000
Testing samples: 14000


In [9]:
n_classes = len(np.unique(np.concatenate([y_train, y_test])))
concept_names = SHAPES3D_CONCEPT_NAMES
concept_values = [len(np.unique(np.concatenate([c_train[:, i], c_test[:, i]]))) for i in range(c_train.shape[1])]

# Split training into validation set as well 
X_train, X_valid = X_train[:40000], X_train[40000:]
y_train, y_valid = y_train[:40000], y_train[40000:]
c_train, c_valid = c_train[:40000], c_train[40000:]

In [5]:
# Load adversarial samples (we will need this for adversarial
# shift).
adv_samples = np.load("../../data/adversarial_samples/X_adversarial_3dshapes.npy")

In [11]:
# Reshape to appropriate shift input
# It is noteworthy that for efficiency, we represent the images as only 2 dimension
# when we preprocessing (number of instances/ batch size * flatten size).
# When visualising back the image, we need to reshape it back to the original dimension
ORIGINAL_SHAPE = X_test.shape[1:] # constant hold the image original shape
X_test_flatten = deepcopy(X_test.reshape(X_test.shape[0], -1))
X_train_flatten = deepcopy(X_train.reshape(X_train.shape[0], -1))
X_valid_flatten = deepcopy(X_valid.reshape(X_valid.shape[0], -1))

## Dimensionality reduction

We implemented various dimensionality reduction methods, amounting to:
- End to end model (label classifiers/ BBSD)
- Concept bottleneck model (CBSD)
- Principal component analysis (PCA)
- Sparse random projection (SRP)

### End-to-end model

In [13]:
path = "../../models/end_to_end_3dshapes_task1"
# For training and saving
# histories, end_to_end_model = end_to_end_neural_network(n_classes, Dataset.SHAPES3D, 
#                          X_train, y_train, X_valid, y_valid, path)

# For loading
end_to_end_model = tf.keras.models.load_model(path)

Epoch 1/200
313/313 [==============================] - 7s 23ms/step - loss: 1.9887 - accuracy: 0.2032 - val_loss: 1.6767 - val_accuracy: 0.3778
Epoch 2/200
313/313 [==============================] - 6s 19ms/step - loss: 1.1458 - accuracy: 0.5017 - val_loss: 0.4289 - val_accuracy: 0.9445
Epoch 3/200
313/313 [==============================] - 6s 20ms/step - loss: 0.5218 - accuracy: 0.7829 - val_loss: 0.0539 - val_accuracy: 0.9973
Epoch 4/200
313/313 [==============================] - 6s 20ms/step - loss: 0.2205 - accuracy: 0.9165 - val_loss: 0.0060 - val_accuracy: 1.0000
Epoch 5/200
313/313 [==============================] - 6s 20ms/step - loss: 0.1153 - accuracy: 0.9580 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 6/200
313/313 [==============================] - 6s 20ms/step - loss: 0.0662 - accuracy: 0.9769 - val_loss: 4.3184e-04 - val_accuracy: 1.0000
Epoch 7/200
313/313 [==============================] - 6s 20ms/step - loss: 0.0474 - accuracy: 0.9841 - val_loss: 6.4257e-04 - val_a

In [14]:
# Evaluate model
y_pred = end_to_end_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1688
         1.0       1.00      1.00      1.00      1704
         2.0       1.00      1.00      1.00      1800
         3.0       1.00      1.00      1.00      1774
         4.0       1.00      1.00      1.00      1748
         5.0       1.00      1.00      1.00      1743
         6.0       1.00      1.00      1.00      1762
         7.0       1.00      1.00      1.00      1781

    accuracy                           1.00     14000
   macro avg       1.00      1.00      1.00     14000
weighted avg       1.00      1.00      1.00     14000



### Concept bottleneck model

**Input to Concept**

In [15]:
path = "../../models/multitask_3dshapes"
# For training and saving
histories, mt_model = multitask_model(Dataset.SHAPES3D,
                                            X_train, c_train,
                                            X_valid, c_valid, path, concept_values)

# For loading
mt_model = tf.keras.models.load_model(path)

Epoch 1/200
313/313 [==============================] - 12s 37ms/step - loss: 11.9473 - floor_loss: 1.3522 - wall_loss: 2.0598 - object_loss: 2.1339 - scale_loss: 2.1776 - shape_loss: 1.3913 - orientation_loss: 2.8326 - floor_accuracy: 0.4512 - wall_accuracy: 0.2561 - object_accuracy: 0.2167 - scale_accuracy: 0.1350 - shape_accuracy: 0.3208 - orientation_accuracy: 0.0688 - val_loss: 10.5097 - val_floor_loss: 1.0413 - val_wall_loss: 1.7010 - val_object_loss: 1.8302 - val_scale_loss: 2.0393 - val_shape_loss: 1.1812 - val_orientation_loss: 2.7167 - val_floor_accuracy: 0.7563 - val_wall_accuracy: 0.5278 - val_object_accuracy: 0.3962 - val_scale_accuracy: 0.1852 - val_shape_accuracy: 0.5375 - val_orientation_accuracy: 0.0850
Epoch 2/200
313/313 [==============================] - 10s 31ms/step - loss: 10.4759 - floor_loss: 0.9373 - wall_loss: 1.5977 - object_loss: 1.7429 - scale_loss: 2.1701 - shape_loss: 1.1480 - orientation_loss: 2.8800 - floor_accuracy: 0.6480 - wall_accuracy: 0.4299 - obj

313/313 [==============================] - 10s 32ms/step - loss: 3.2841 - floor_loss: 0.3228 - wall_loss: 0.5372 - object_loss: 0.6690 - scale_loss: 0.7013 - shape_loss: 0.3222 - orientation_loss: 0.7316 - floor_accuracy: 0.8842 - wall_accuracy: 0.8109 - object_accuracy: 0.7427 - scale_accuracy: 0.7291 - shape_accuracy: 0.8786 - orientation_accuracy: 0.7431 - val_loss: 0.6012 - val_floor_loss: 0.0148 - val_wall_loss: 0.0547 - val_object_loss: 0.1708 - val_scale_loss: 0.1973 - val_shape_loss: 0.0194 - val_orientation_loss: 0.1443 - val_floor_accuracy: 1.0000 - val_wall_accuracy: 0.9988 - val_object_accuracy: 0.9915 - val_scale_accuracy: 0.9967 - val_shape_accuracy: 1.0000 - val_orientation_accuracy: 0.9947
Epoch 24/200
313/313 [==============================] - 10s 31ms/step - loss: 3.2522 - floor_loss: 0.3292 - wall_loss: 0.5323 - object_loss: 0.6608 - scale_loss: 0.6828 - shape_loss: 0.3198 - orientation_loss: 0.7273 - floor_accuracy: 0.8816 - wall_accuracy: 0.8110 - object_accuracy: 

313/313 [==============================] - 9s 30ms/step - loss: 2.7243 - floor_loss: 0.2937 - wall_loss: 0.4662 - object_loss: 0.5658 - scale_loss: 0.5357 - shape_loss: 0.2846 - orientation_loss: 0.5783 - floor_accuracy: 0.8959 - wall_accuracy: 0.8386 - object_accuracy: 0.7902 - scale_accuracy: 0.7960 - shape_accuracy: 0.8944 - orientation_accuracy: 0.8011 - val_loss: 0.3003 - val_floor_loss: 0.0100 - val_wall_loss: 0.0266 - val_object_loss: 0.1050 - val_scale_loss: 0.0951 - val_shape_loss: 0.0107 - val_orientation_loss: 0.0529 - val_floor_accuracy: 1.0000 - val_wall_accuracy: 1.0000 - val_object_accuracy: 0.9997 - val_scale_accuracy: 1.0000 - val_shape_accuracy: 1.0000 - val_orientation_accuracy: 1.0000
Epoch 46/200
313/313 [==============================] - 9s 28ms/step - loss: 2.7049 - floor_loss: 0.2944 - wall_loss: 0.4560 - object_loss: 0.5627 - scale_loss: 0.5360 - shape_loss: 0.2865 - orientation_loss: 0.5693 - floor_accuracy: 0.8931 - wall_accuracy: 0.8418 - object_accuracy: 0.

313/313 [==============================] - 10s 32ms/step - loss: 2.4045 - floor_loss: 0.2596 - wall_loss: 0.4190 - object_loss: 0.5008 - scale_loss: 0.4613 - shape_loss: 0.2638 - orientation_loss: 0.4999 - floor_accuracy: 0.9061 - wall_accuracy: 0.8551 - object_accuracy: 0.8182 - scale_accuracy: 0.8268 - shape_accuracy: 0.9039 - orientation_accuracy: 0.8296 - val_loss: 0.1839 - val_floor_loss: 0.0109 - val_wall_loss: 0.0206 - val_object_loss: 0.0628 - val_scale_loss: 0.0525 - val_shape_loss: 0.0092 - val_orientation_loss: 0.0278 - val_floor_accuracy: 1.0000 - val_wall_accuracy: 1.0000 - val_object_accuracy: 1.0000 - val_scale_accuracy: 1.0000 - val_shape_accuracy: 1.0000 - val_orientation_accuracy: 1.0000
Epoch 68/200
313/313 [==============================] - 9s 29ms/step - loss: 2.3618 - floor_loss: 0.2572 - wall_loss: 0.4146 - object_loss: 0.4835 - scale_loss: 0.4625 - shape_loss: 0.2598 - orientation_loss: 0.4844 - floor_accuracy: 0.9077 - wall_accuracy: 0.8558 - object_accuracy: 0

313/313 [==============================] - 9s 29ms/step - loss: 2.1121 - floor_loss: 0.2367 - wall_loss: 0.3772 - object_loss: 0.4184 - scale_loss: 0.4005 - shape_loss: 0.2344 - orientation_loss: 0.4449 - floor_accuracy: 0.9179 - wall_accuracy: 0.8693 - object_accuracy: 0.8514 - scale_accuracy: 0.8511 - shape_accuracy: 0.9139 - orientation_accuracy: 0.8493 - val_loss: 0.1084 - val_floor_loss: 0.0093 - val_wall_loss: 0.0121 - val_object_loss: 0.0356 - val_scale_loss: 0.0307 - val_shape_loss: 0.0068 - val_orientation_loss: 0.0139 - val_floor_accuracy: 1.0000 - val_wall_accuracy: 1.0000 - val_object_accuracy: 1.0000 - val_scale_accuracy: 1.0000 - val_shape_accuracy: 1.0000 - val_orientation_accuracy: 1.0000
Epoch 90/200
313/313 [==============================] - 9s 30ms/step - loss: 2.0971 - floor_loss: 0.2352 - wall_loss: 0.3703 - object_loss: 0.4208 - scale_loss: 0.3913 - shape_loss: 0.2378 - orientation_loss: 0.4418 - floor_accuracy: 0.9172 - wall_accuracy: 0.8713 - object_accuracy: 0.

313/313 [==============================] - 10s 31ms/step - loss: 1.8356 - floor_loss: 0.2131 - wall_loss: 0.3286 - object_loss: 0.3594 - scale_loss: 0.3428 - shape_loss: 0.2068 - orientation_loss: 0.3848 - floor_accuracy: 0.9286 - wall_accuracy: 0.8854 - object_accuracy: 0.8748 - scale_accuracy: 0.8753 - shape_accuracy: 0.9250 - orientation_accuracy: 0.8690 - val_loss: 0.0787 - val_floor_loss: 0.0114 - val_wall_loss: 0.0101 - val_object_loss: 0.0239 - val_scale_loss: 0.0202 - val_shape_loss: 0.0052 - val_orientation_loss: 0.0079 - val_floor_accuracy: 1.0000 - val_wall_accuracy: 1.0000 - val_object_accuracy: 1.0000 - val_scale_accuracy: 1.0000 - val_shape_accuracy: 1.0000 - val_orientation_accuracy: 1.0000
Epoch 112/200
313/313 [==============================] - 9s 30ms/step - loss: 1.8563 - floor_loss: 0.2163 - wall_loss: 0.3319 - object_loss: 0.3570 - scale_loss: 0.3478 - shape_loss: 0.2119 - orientation_loss: 0.3914 - floor_accuracy: 0.9273 - wall_accuracy: 0.8857 - object_accuracy: 

313/313 [==============================] - 9s 30ms/step - loss: 1.6269 - floor_loss: 0.1915 - wall_loss: 0.2951 - object_loss: 0.3140 - scale_loss: 0.3055 - shape_loss: 0.1829 - orientation_loss: 0.3379 - floor_accuracy: 0.9374 - wall_accuracy: 0.8990 - object_accuracy: 0.8927 - scale_accuracy: 0.8868 - shape_accuracy: 0.9333 - orientation_accuracy: 0.8855 - val_loss: 0.0506 - val_floor_loss: 0.0087 - val_wall_loss: 0.0059 - val_object_loss: 0.0155 - val_scale_loss: 0.0122 - val_shape_loss: 0.0034 - val_orientation_loss: 0.0049 - val_floor_accuracy: 1.0000 - val_wall_accuracy: 1.0000 - val_object_accuracy: 1.0000 - val_scale_accuracy: 1.0000 - val_shape_accuracy: 1.0000 - val_orientation_accuracy: 1.0000
Epoch 134/200
313/313 [==============================] - 9s 30ms/step - loss: 1.6367 - floor_loss: 0.1967 - wall_loss: 0.2947 - object_loss: 0.3132 - scale_loss: 0.3076 - shape_loss: 0.1875 - orientation_loss: 0.3371 - floor_accuracy: 0.9360 - wall_accuracy: 0.8970 - object_accuracy: 0

In [16]:
# Evaluate model
for i, pred in enumerate(mt_model.predict(X_test)):
    print("*"*20, f"Model: {SHAPES3D_CONCEPT_NAMES[i]}", "*"*20)
    c_truth = c_test[:, i]
    c_pred = np.argmax(pred, axis=1)
    
    print(classification_report(c_truth, c_pred))
    print("\n\n")

******************** Model: floor ********************
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3149
         1.0       1.00      1.00      1.00      3357
         2.0       1.00      1.00      1.00      3356
         3.0       1.00      1.00      1.00      3308
         4.0       1.00      1.00      1.00       830

    accuracy                           1.00     14000
   macro avg       1.00      1.00      1.00     14000
weighted avg       1.00      1.00      1.00     14000




******************** Model: wall ********************
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1397
         1.0       1.00      1.00      1.00      1671
         2.0       1.00      1.00      1.00      1496
         3.0       1.00      1.00      1.00      1353
         4.0       1.00      1.00      1.00      1363
         5.0       1.00      1.00      1.00      1368
         6.0       1

**Concept to Output**

In [17]:
# Build and train model. For simplicity, we used logistic regression
# although can be substituted using other model.
com = LogisticRegression()
com.fit(c_train, y_train)

LogisticRegression()

In [18]:
y_test_pred = com.predict(c_test)
print(classification_report(y_test_pred, y_test))
print(confusion_matrix(y_test_pred, y_test))

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      1659
         1.0       0.85      0.92      0.88      1588
         2.0       0.91      0.84      0.88      1938
         3.0       0.83      0.88      0.86      1685
         4.0       0.84      0.79      0.81      1864
         5.0       0.91      0.92      0.92      1722
         6.0       0.99      1.00      0.99      1748
         7.0       1.00      0.99      1.00      1796

    accuracy                           0.92     14000
   macro avg       0.92      0.92      0.92     14000
weighted avg       0.92      0.92      0.91     14000

[[1657    2    0    0    0    0    0    0]
 [  31 1456  101    0    0    0    0    0]
 [   0  246 1637   55    0    0    0    0]
 [   0    0   62 1479  144    0    0    0]
 [   0    0    0  240 1471  153    0    0]
 [   0    0    0    0  133 1588    1    0]
 [   0    0    0    0    0    2 1746    0]
 [   0    0    0    0    0    0   15 1781]]


### Principal component analysis

In [ ]:
pca, n_components = principal_components_analysis(X_train_flatten)
print(f"The number of components to explain 80% of variance is {n_components}.")

### Sparse random projection

In [ ]:
srp, n_components = sparse_random_projection(X_train_flatten)
print(f"The number of components to explain 80% of variance is {n_components}.")

## Data collection

This section performs various experiments to collect data. We consider various dimensionality reduced methods discussed in the paper and thesis.

### PCA

In [7]:
method = DimensionalityReductor.PCA
model = pca
method_str = "PCA"

#### Knockout shift

In [8]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko"

In [14]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Concept shifts

We consider the following concept shift combinations:
- floor
- wall
- object

In [ ]:
shift_type = ShiftType.Concept

list_shift_str = [
    "floor",
    "wall",
    "object_scale",
]

list_shift_type_params = [
    {"cl": MAJORITY, "concept_idx": 0}, # scale is index 2 in the concept names
    {"cl": MAJORITY, "concept_idx": 1},
    [{"cl": MAJORITY, "concept_idx": 2}, {"cl": MAJORITY, "concept_idx": 3}],
]

In [ ]:
for shift_str, shift_type_params in tqdm(zip(list_shift_str, list_shift_type_params)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Image shifts

We consider the following image shifts:
- rotation
- shear
- flip
- all (combination of image shifts)

In [2]:
list_shift = [
    ShiftType.Rotation,
    ShiftType.Shear,
    ShiftType.Flip,
    ShiftType.All
]

list_shift_str = [
    "rotation",
    "shear",
    "flip",
    "all"
]

shift_type_param = {"orig_dims": ORIGINAL_SHAPE}

In [ ]:
for shift_str, shift_type in tqdm(zip(list_shift_str, list_shift)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_param, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Gaussian shift

In [ ]:
shift_type = ShiftType.Gaussian
shift_type_params = None
shift_str = "gaussian"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Adversarial shift

In [ ]:
shift_type = ShiftType.Adversarial
shift_type_params = {
    "adv_samples": adv_samples
}
shift_str = "adversarial"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

### SRP

In [36]:
method = DimensionalityReductor.SRP
model = srp
method_str = "SRP"

#### Knockout shift

In [8]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko"

In [14]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Concept shifts

In [ ]:
shift_type = ShiftType.Concept

list_shift_str = [
    "floor",
    "wall",
    "object_scale",
]

list_shift_type_params = [
    {"cl": MAJORITY, "concept_idx": 0}, # scale is index 2 in the concept names
    {"cl": MAJORITY, "concept_idx": 1},
    [{"cl": MAJORITY, "concept_idx": 2}, {"cl": MAJORITY, "concept_idx": 3}],
]

In [ ]:
for shift_str, shift_type_params in tqdm(zip(list_shift_str, list_shift_type_params)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Image shifts


In [2]:
list_shift = [
    ShiftType.Rotation,
    ShiftType.Shear,
    ShiftType.Flip,
    ShiftType.All
]

list_shift_str = [
    "rotation",
    "shear",
    "flip",
    "all"
]

shift_type_param = {"orig_dims": ORIGINAL_SHAPE}

In [ ]:
for shift_str, shift_type in tqdm(zip(list_shift_str, list_shift)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_param, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Gaussian shift

In [ ]:
shift_type = ShiftType.Gaussian
shift_type_params = None
shift_str = "gaussian"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Adversarial shift

In [ ]:
shift_type = ShiftType.Adversarial
shift_type_params = {
    "adv_samples": adv_samples
}
shift_str = "adversarial"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

### BBSDs

In [36]:
method = DimensionalityReductor.BBSDs
model = end_to_end_model
method_str = "BBSDs"

#### Knockout shift

In [8]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko"

In [14]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Concept shifts

In [ ]:
shift_type = ShiftType.Concept

list_shift_str = [
    "floor",
    "wall",
    "object_scale",
]

list_shift_type_params = [
    {"cl": MAJORITY, "concept_idx": 0}, # scale is index 2 in the concept names
    {"cl": MAJORITY, "concept_idx": 1},
    [{"cl": MAJORITY, "concept_idx": 2}, {"cl": MAJORITY, "concept_idx": 3}],
]

In [ ]:
for shift_str, shift_type_params in tqdm(zip(list_shift_str, list_shift_type_params)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Image shifts


In [2]:
list_shift = [
    ShiftType.Rotation,
    ShiftType.Shear,
    ShiftType.Flip,
    ShiftType.All
]

list_shift_str = [
    "rotation",
    "shear",
    "flip",
    "all"
]

shift_type_param = {"orig_dims": ORIGINAL_SHAPE}

In [ ]:
for shift_str, shift_type in tqdm(zip(list_shift_str, list_shift)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_param, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Gaussian shift

In [ ]:
shift_type = ShiftType.Gaussian
shift_type_params = None
shift_str = "gaussian"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Adversarial shift

In [ ]:
shift_type = ShiftType.Adversarial
shift_type_params = {
    "adv_samples": adv_samples
}
shift_str = "adversarial"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

### BBSDh

In [36]:
method = DimensionalityReductor.BBSDh
model = end_to_end_model
method_str = "BBSDh"

#### Knockout shift

In [8]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko"

In [14]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Concept shifts

In [ ]:
shift_type = ShiftType.Concept

list_shift_str = [
    "floor",
    "wall",
    "object_scale",
]

list_shift_type_params = [
    {"cl": MAJORITY, "concept_idx": 0}, # scale is index 2 in the concept names
    {"cl": MAJORITY, "concept_idx": 1},
    [{"cl": MAJORITY, "concept_idx": 2}, {"cl": MAJORITY, "concept_idx": 3}],
]

In [ ]:
for shift_str, shift_type_params in tqdm(zip(list_shift_str, list_shift_type_params)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Image shifts


In [2]:
list_shift = [
    ShiftType.Rotation,
    ShiftType.Shear,
    ShiftType.Flip,
    ShiftType.All
]

list_shift_str = [
    "rotation",
    "shear",
    "flip",
    "all"
]

shift_type_param = {"orig_dims": ORIGINAL_SHAPE}

In [ ]:
for shift_str, shift_type in tqdm(zip(list_shift_str, list_shift)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_param, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Gaussian shift

In [ ]:
shift_type = ShiftType.Gaussian
shift_type_params = None
shift_str = "gaussian"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Adversarial shift

In [ ]:
shift_type = ShiftType.Adversarial
shift_type_params = {
    "adv_samples": adv_samples
}
shift_str = "adversarial"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

### CBSDs

In [36]:
method = DimensionalityReductor.CBSDs
model = mt_model
method_str = "CBSDs"

#### Knockout shift

In [8]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko"

In [14]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Concept shifts

In [ ]:
shift_type = ShiftType.Concept

list_shift_str = [
    "floor",
    "wall",
    "object_scale",
]

list_shift_type_params = [
    {"cl": MAJORITY, "concept_idx": 0}, # scale is index 2 in the concept names
    {"cl": MAJORITY, "concept_idx": 1},
    [{"cl": MAJORITY, "concept_idx": 2}, {"cl": MAJORITY, "concept_idx": 3}],
]

In [ ]:
for shift_str, shift_type_params in tqdm(zip(list_shift_str, list_shift_type_params)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Image shifts


In [2]:
list_shift = [
    ShiftType.Rotation,
    ShiftType.Shear,
    ShiftType.Flip,
    ShiftType.All
]

list_shift_str = [
    "rotation",
    "shear",
    "flip",
    "all"
]

shift_type_param = {"orig_dims": ORIGINAL_SHAPE}

In [ ]:
for shift_str, shift_type in tqdm(zip(list_shift_str, list_shift)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_param, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Gaussian shift

In [ ]:
shift_type = ShiftType.Gaussian
shift_type_params = None
shift_str = "gaussian"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Adversarial shift

In [ ]:
shift_type = ShiftType.Adversarial
shift_type_params = {
    "adv_samples": adv_samples
}
shift_str = "adversarial"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

### CBSDh

In [36]:
method = DimensionalityReductor.CBSDh
model = mt_model
method_str = "CBSDh"

#### Knockout shift

In [8]:
shift_type = ShiftType.Knockout
shift_type_params = {"cl": MAJORITY}
shift_str = "ko"

In [14]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Concept shifts

In [ ]:
shift_type = ShiftType.Concept

list_shift_str = [
    "floor",
    "wall",
    "object_scale",
]

list_shift_type_params = [
    {"cl": MAJORITY, "concept_idx": 0}, # scale is index 2 in the concept names
    {"cl": MAJORITY, "concept_idx": 1},
    [{"cl": MAJORITY, "concept_idx": 2}, {"cl": MAJORITY, "concept_idx": 3}],
]

In [ ]:
for shift_str, shift_type_params in tqdm(zip(list_shift_str, list_shift_type_params)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Image shifts


In [2]:
list_shift = [
    ShiftType.Rotation,
    ShiftType.Shear,
    ShiftType.Flip,
    ShiftType.All
]

list_shift_str = [
    "rotation",
    "shear",
    "flip",
    "all"
]

shift_type_param = {"orig_dims": ORIGINAL_SHAPE}

In [ ]:
for shift_str, shift_type in tqdm(zip(list_shift_str, list_shift)):
    dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_param, n_exp=50, n_std=2)
    
    # Save
    save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Gaussian shift

In [ ]:
shift_type = ShiftType.Gaussian
shift_type_params = None
shift_str = "gaussian"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")

#### Adversarial shift

In [ ]:
shift_type = ShiftType.Adversarial
shift_type_params = {
    "adv_samples": adv_samples
}
shift_str = "adversarial"

In [ ]:
dict_result = main_experiment(model, method, X_valid, y_valid,
                             c_valid, X_test_flatten, y_test, c_test,
                             shift_type, ORIGINAL_SHAPE, n_classes,
                             concept_names, concept_values, 
                             shift_type_params, n_exp=50, n_std=2)

In [ ]:
# Save file
save_result(shift_str, method_str, dict_result, True, "3dshapes")